# Loading Data

In [3]:
import matplotlib.pyplot as plt

In [18]:
import datetime as dt

In [4]:
import gc
from warnings import filterwarnings

In [5]:
import pandas as pd
import numpy as np

In [6]:
pd.set_option('display.max_rows', 80)

In [5]:
session = pd.read_pickle('../../session_roll_agg.pkl')
events = pd.read_pickle('../../events_roll_agg.pkl')

In [6]:
df = session

In [95]:
user_date = pd.read_pickle('../../user_date.pkl')
user_date = user_date.groupby('user_id_hash').user_created_timestamp.max().to_frame()

In [96]:
user_date.reset_index(inplace=True)

In [97]:
user_date.head()

,user_id_hash,user_created_timestamp
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,2018-11-10
1,000059859ec188af6035870faf885c3038cedda05b3a54...,2018-11-22
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,2018-10-07
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,2018-10-18
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,2018-11-22


In [7]:
filterwarnings('ignore')

In [98]:
user_date.shape

(619519, 2)

In [99]:
df = df.merge(user_date, on=['user_id_hash'], how='left')

In [100]:
del user_date

In [101]:
gc.collect()

1353

In [15]:
purchase = events[events.event == '8'].copy(deep=True)

In [16]:
purchase = purchase.loc[:, ['anchor_date', 'user_id_hash',
                            'event_count', 'event_sum']]

In [17]:
purchase.columns

Index(['anchor_date', 'user_id_hash', 'event_count', 'event_sum'], dtype='object')

In [18]:
purchase.columns = ['anchor_date', 'user_id_hash',
                    'no_of_buys', 'sum_of_buys']

In [19]:
purchase.reset_index(inplace=True, drop=True)

In [20]:
purchase.head()

,anchor_date,user_id_hash,no_of_buys,sum_of_buys
0,2018-10-01,001c2d06972b17ee6a075ff9fbdae33eccb0bbf28357ea...,1,1.393000
1,2018-10-01,0026497d2043c8530220eeb8e66051982cc6df3139a9b0...,3,10.478999
2,2018-10-01,006b49314baf054c636e7448d4f9b0df046aae9e9c17d0...,10,20.229999
3,2018-10-01,007327aa3a27718c76e6cd73c3971304280c8856f62940...,1,1.393000
4,2018-10-01,0088d9be17b2ec4b3621150117a0ede815547d4241eae1...,2,6.986000


In [21]:
purchase.sort_values('anchor_date', inplace=True)

In [23]:
purchase1 = purchase.copy(deep=True)
purchase1.anchor_date = purchase1.anchor_date.apply(lambda x: x- dt.timedelta(days=7))
purchase1.sort_values('anchor_date', inplace=True)

In [24]:
purchase2 = purchase.copy(deep=True)
purchase2.anchor_date = purchase2.anchor_date.apply(lambda x: x- dt.timedelta(days=14))
purchase2.sort_values('anchor_date', inplace=True)

In [25]:
df.anchor_date = df.anchor_date.astype('category')

In [26]:
purchase.anchor_date = purchase.anchor_date.astype('category')
purchase1.anchor_date = purchase1.anchor_date.astype('category')
purchase2.anchor_date = purchase2.anchor_date.astype('category')

In [27]:
df.shape

(6976625, 14)

In [28]:
df = df.merge(purchase, on=['user_id_hash', 'anchor_date'], how='outer')

In [29]:
df.shape

(6976994, 16)

In [30]:
df.no_of_buys.fillna(0, inplace=True)
df.sum_of_buys.fillna(0, inplace=True)

In [31]:
df = df.merge(purchase1, on=['user_id_hash', 'anchor_date'], how='outer')
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y1'}, inplace=True)

In [32]:
df.shape

(7163335, 17)

In [33]:
df.anchor_date = df.anchor_date.astype('category')

In [34]:
df = df.merge(purchase2, on=['user_id_hash', 'anchor_date'], how='outer')
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y2'}, inplace=True)

In [35]:
df.shape

(7391174, 18)

In [36]:
df.head()

,anchor_date,user_id_hash,no_sessions,no_city,last_city,last_country,last_region,region_count,last_locale,total_session_time,mean_session_time,median_session_time,last_lat_long,user_created_timestamp,no_of_buys,sum_of_buys,y1,y2
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,5.0,1.0,unk,US,ca,1.0,en_US,4730609.0,946121.8,1176899.0,"(37.28716659545898, -121.94995880126952)",2018-10-07,0.0,0.0,NaN,NaN
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2.0,1.0,unk,US,ga,1.0,en_US,403432.0,201716.0,201716.0,"(33.751495361328125, -84.74771118164062)",2018-10-05,0.0,0.0,NaN,NaN
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,2.0,1.0,unk,US,ky,1.0,en_US,150138.0,75069.0,75069.0,"(36.60725784301758, -83.71428680419922)",2018-10-03,0.0,0.0,NaN,NaN
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1.0,1.0,omaha,US,ne,1.0,en_US,0.0,0.0,0.0,"(41.25653839111328, -95.93450164794922)",2018-10-02,0.0,0.0,NaN,NaN
4,2018-10-01,00059c820dca1ccadfd45361c40f68b656df8a5212b993...,1.0,1.0,rio de janeiro,BR,rj,1.0,pt-BR_BR,123131.0,123131.0,123131.0,"(-22.90684700012207, -43.17289733886719)",2018-10-01,0.0,0.0,NaN,NaN


In [37]:
df['y1'].fillna(0, inplace=True)
df['y2'].fillna(0, inplace=True)
df['y2'] = df['y1'] + df['y2']
df['y1'] = df['y1'] > 0
df['y2'] = df['y2'] > 0

In [38]:
df.columns.values

array(['anchor_date', 'user_id_hash', 'no_sessions', 'no_city',
       'last_city', 'last_country', 'last_region', 'region_count',
       'last_locale', 'total_session_time', 'mean_session_time',
       'median_session_time', 'last_lat_long', 'user_created_timestamp',
       'no_of_buys', 'sum_of_buys', 'y1', 'y2'], dtype=object)

In [39]:
df.isna().sum()

anchor_date                    0
user_id_hash                   0
no_sessions               414549
no_city                   414549
last_city                 417348
last_country              421149
last_region               417378
region_count              414549
last_locale               414549
total_session_time        414549
mean_session_time         414549
median_session_time       414549
last_lat_long             414549
user_created_timestamp    414549
no_of_buys                414180
sum_of_buys               414180
y1                             0
y2                             0
dtype: int64

In [40]:
df.drop(['last_lat_long'], inplace=True, axis=1)

In [41]:
df.user_id_hash = df.user_id_hash.astype('category')
df.last_city = df.last_city.astype('category')
df.last_country = df.last_country.astype('category')
df.last_region = df.last_region.astype('category')
df.last_locale = df.last_locale.astype('category')

In [102]:
df['user_age'] = (df.anchor_date.dt.date - df.user_created_timestamp)\
                    .astype('timedelta64[D]')

In [103]:
df.drop(['user_created_timestamp'], inplace=True, axis=1)

In [44]:
# df.to_pickle('../mergedv3.pkl')

In [45]:
# df = pd.read_pickle('../mergedv3.pkl')

In [46]:
df.anchor_date.min()

Timestamp('2018-09-11 00:00:00')

In [47]:
df.anchor_date.max()

Timestamp('2018-12-14 00:00:00')

In [48]:
df.reset_index(inplace=True, drop=True)

In [49]:
anchor_dates = df.anchor_date.unique()

In [50]:
anchor_number = {str(a)[:10]: sum(df.anchor_date == a) for a in sorted(anchor_dates)}

In [51]:
x = dt.date(2018, 12, 8)
x

datetime.date(2018, 12, 8)

# Multiple weeks

In [52]:
[x - dt.timedelta(days=7*i) for i in range(11)]

[datetime.date(2018, 12, 8),
 datetime.date(2018, 12, 1),
 datetime.date(2018, 11, 24),
 datetime.date(2018, 11, 17),
 datetime.date(2018, 11, 10),
 datetime.date(2018, 11, 3),
 datetime.date(2018, 10, 27),
 datetime.date(2018, 10, 20),
 datetime.date(2018, 10, 13),
 datetime.date(2018, 10, 6),
 datetime.date(2018, 9, 29)]

In [53]:
df.anchor_date.isin([x - dt.timedelta(days=7*i) for i in range(11)]).sum()

1043065

In [54]:
df.shape

(7391174, 17)

In [55]:
df = df[df.anchor_date.isin([x - dt.timedelta(days=7*i) for i in range(11)])]

In [56]:
df.shape

(1043065, 17)

In [57]:
df.isna().sum(axis=0)

anchor_date                0
user_id_hash               0
no_sessions            52744
no_city                52744
last_city              53140
last_country           53691
last_region            53145
region_count           52744
last_locale            52744
total_session_time     52744
mean_session_time      52744
median_session_time    52744
no_of_buys             52700
sum_of_buys            52700
y1                         0
y2                         0
user_age               52744
dtype: int64

In [58]:
df.reset_index(inplace=True, drop=True)

In [59]:
from warnings import filterwarnings

In [29]:
filterwarnings('ignore')

In [61]:
df['y1'] = df['y1'].astype(np.int)
df['y2'] = df['y2'].astype(np.int)

In [62]:
df.last_city = df.last_city.cat.codes
df.last_country = df.last_country.cat.codes
df.last_region = df.last_region.cat.codes
df.last_locale = df.last_locale.cat.codes

In [63]:
df.head()

,anchor_date,user_id_hash,no_sessions,no_city,last_city,last_country,last_region,region_count,last_locale,total_session_time,mean_session_time,median_session_time,no_of_buys,sum_of_buys,y1,y2,user_age
0,2018-09-29,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2.0,1.0,618,211,117,1.0,66,403432.0,2.017160e+05,201716.0,0.0,0.0,0,0,-6.0
1,2018-09-29,0003f8bda56230a49445880e559b718e94ba37344494a7...,1.0,1.0,618,211,157,1.0,66,0.0,0.000000e+00,0.0,0.0,0.0,0,0,-4.0
2,2018-09-29,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1.0,1.0,448,211,189,1.0,66,0.0,0.000000e+00,0.0,0.0,0.0,0,0,-3.0
3,2018-09-29,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,3.0,1.0,355,211,85,1.0,66,6173560.0,2.057853e+06,2691647.0,0.0,0.0,0,0,-6.0
4,2018-09-29,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...,1.0,1.0,535,224,122,1.0,54,0.0,0.000000e+00,0.0,0.0,0.0,0,0,-3.0


In [64]:
df_bkp = df.copy(deep=True)

In [65]:
df = df_bkp.copy(deep=True)

In [66]:
df.columns

Index(['anchor_date', 'user_id_hash', 'no_sessions', 'no_city', 'last_city',
       'last_country', 'last_region', 'region_count', 'last_locale',
       'total_session_time', 'mean_session_time', 'median_session_time',
       'no_of_buys', 'sum_of_buys', 'y1', 'y2', 'user_age'],
      dtype='object')

don't send in y1 and y2

In [67]:
merge_cols = ['anchor_date', 'user_id_hash', 'no_sessions', 'no_city',
              'last_city', 'last_country', 'last_region', 'region_count',
              'last_locale', 'total_session_time', 'mean_session_time',
              'median_session_time', 'no_of_buys', 'sum_of_buys']

In [68]:
for wk in range(1, 5):
    print(wk)
    temp = df.copy(deep=True)
    temp.anchor_date = \
        temp.anchor_date.apply(lambda x: x + dt.timedelta(days=(7*wk)))
    df = df.merge(temp[merge_cols].\
                  rename(columns={i: f'pst_wk{wk}_{i}' 
                                  for i in merge_cols[2:]}),
              on=['user_id_hash', 'anchor_date'],
              how='outer')

1
2
3
4


In [69]:
df.shape

(7351039, 65)

In [70]:
df_bkp.shape

(1043065, 17)

In [71]:
df.tail().T

,7351034,7351035,7351036,7351037,7351038
anchor_date,2019-02-02 00:00:00,2019-02-02 00:00:00,2019-02-09 00:00:00,2019-02-16 00:00:00,2019-02-16 00:00:00
user_id_hash,c59a17f2d964aa477c4fbea6065382000ee9689de5946f...,b8b0f7687859a539ffc26bc71ed2a377b38450f4d6ad4c...,ad7aac6af49f2578d80161800fb36d4fe109d0fbcc1b53...,c58945d5e958f5a2bb805243b7c13d5bf9a671f9f95287...,d6eb2164eccc438deafb485c67ea2f499539746a50ac90...
no_sessions,NaN,NaN,NaN,NaN,NaN
no_city,NaN,NaN,NaN,NaN,NaN
last_city,NaN,NaN,NaN,NaN,NaN
last_country,NaN,NaN,NaN,NaN,NaN
last_region,NaN,NaN,NaN,NaN,NaN
region_count,NaN,NaN,NaN,NaN,NaN
last_locale,NaN,NaN,NaN,NaN,NaN
total_session_time,NaN,NaN,NaN,NaN,NaN


In [72]:
df.isna().sum()

anchor_date                          0
user_id_hash                         0
no_sessions                    6360718
no_city                        6360718
last_city                      6307974
last_country                   6307974
last_region                    6307974
region_count                   6360718
last_locale                    6307974
total_session_time             6360718
mean_session_time              6360718
median_session_time            6360718
no_of_buys                     6360674
sum_of_buys                    6360674
y1                             6307974
y2                             6307974
user_age                       6360718
pst_wk1_no_sessions            6360718
pst_wk1_no_city                6360718
pst_wk1_last_city              6307974
pst_wk1_last_country           6307974
pst_wk1_last_region            6307974
pst_wk1_region_count           6360718
pst_wk1_last_locale            6307974
pst_wk1_total_session_time     6360718
pst_wk1_mean_session_time

In [73]:
df['y1'].fillna(0, inplace=True)
df['y2'].fillna(0, inplace=True)

In [74]:
df['no_nan'] = df.isna().sum(axis=1)

In [75]:
df.shape

(7351039, 66)

In [76]:
df.no_nan.value_counts()

61    3724858
49    2049644
48     598151
37     348204
36     159175
25     103427
24      71705
0       46289
12      39374
13      33505
57      28943
40      20066
53      19956
41      14235
28      13850
33      12670
45      10789
44       9056
32       8824
8        8749
16       8269
21       6981
20       6804
29       6530
17        422
9         343
55         50
47         33
43         28
39         28
35         22
51         19
31         13
27         10
15          4
14          3
22          2
7           2
18          1
19          1
30          1
26          1
6           1
23          1
Name: no_nan, dtype: int64

In [77]:
df = df[df.no_nan != 60]

In [78]:
df.no_nan.value_counts()

61    3724858
49    2049644
48     598151
37     348204
36     159175
25     103427
24      71705
0       46289
12      39374
13      33505
57      28943
40      20066
53      19956
41      14235
28      13850
33      12670
45      10789
44       9056
32       8824
8        8749
16       8269
21       6981
20       6804
29       6530
17        422
9         343
55         50
47         33
43         28
39         28
35         22
51         19
31         13
27         10
15          4
14          3
22          2
7           2
18          1
19          1
30          1
26          1
6           1
23          1
Name: no_nan, dtype: int64

In [79]:
df[df.no_nan == 48].head().T

,0,1,2,3,4
anchor_date,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00
user_id_hash,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,0003f8bda56230a49445880e559b718e94ba37344494a7...,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...
no_sessions,2,1,1,3,1
no_city,1,1,1,1,1
last_city,618,618,448,355,535
last_country,211,211,211,211,224
last_region,117,157,189,85,122
region_count,1,1,1,1,1
last_locale,66,66,66,66,54
total_session_time,403432,0,0,6.17356e+06,0


In [80]:
df.drop(['no_nan'], inplace=True, axis=1)

In [81]:
t = df.isna().sum()
t[t > 0]

no_sessions                    6360718
no_city                        6360718
last_city                      6307974
last_country                   6307974
last_region                    6307974
region_count                   6360718
last_locale                    6307974
total_session_time             6360718
mean_session_time              6360718
median_session_time            6360718
no_of_buys                     6360674
sum_of_buys                    6360674
user_age                       6360718
pst_wk1_no_sessions            6360718
pst_wk1_no_city                6360718
pst_wk1_last_city              6307974
pst_wk1_last_country           6307974
pst_wk1_last_region            6307974
pst_wk1_region_count           6360718
pst_wk1_last_locale            6307974
pst_wk1_total_session_time     6360718
pst_wk1_mean_session_time      6360718
pst_wk1_median_session_time    6360718
pst_wk1_no_of_buys             6360674
pst_wk1_sum_of_buys            6360674
pst_wk2_no_sessions      

In [82]:
# df = df[~df.last_city.isna()]
# df = df[~df.last_country.isna()]
# df = df[~df.last_region.isna()]

In [83]:
# df.total_session_time.fillna(0, inplace=True)
# df.median_session_time.fillna(0, inplace=True)
# df.mean_session_time.fillna(0, inplace=True)
# df.no_sessions.fillna(0, inplace=True)
# df.sum_of_buys.fillna(0, inplace=True)
# df.no_of_buys.fillna(0, inplace=True)

In [84]:
df['t_total_session_time'] = df.total_session_time.fillna(0) + \
        df.pst_wk1_total_session_time.fillna(0) + \
        df.pst_wk2_total_session_time.fillna(0) + \
        df.pst_wk3_total_session_time.fillna(0) + \
        df.pst_wk4_total_session_time.fillna(0)

In [85]:
df['t_no_sessions'] = df.no_sessions.fillna(0) + \
        df.pst_wk1_no_sessions.fillna(0) + \
        df.pst_wk2_no_sessions.fillna(0) + \
        df.pst_wk3_no_sessions.fillna(0) + \
        df.pst_wk4_no_sessions.fillna(0)

In [86]:
df['t_sum_of_buys'] = df.sum_of_buys.fillna(0) + \
        df.pst_wk1_sum_of_buys.fillna(0) + \
        df.pst_wk2_sum_of_buys.fillna(0) + \
        df.pst_wk3_sum_of_buys.fillna(0) + \
        df.pst_wk4_sum_of_buys.fillna(0)

In [87]:
df['t_no_of_buys'] = df.no_of_buys.fillna(0) + \
        df.pst_wk1_no_of_buys.fillna(0) + \
        df.pst_wk2_no_of_buys.fillna(0) + \
        df.pst_wk3_no_of_buys.fillna(0) + \
        df.pst_wk4_no_of_buys.fillna(0)

In [88]:
df.t_total_session_time.isna().sum()

0

stop and think

In [89]:
replace_list = ['no_sessions', 'no_city', 'region_count', 
                'total_session_time', 'mean_session_time',
                'median_session_time', 'no_of_buys', 'sum_of_buys',
                'pst_wk1_no_sessions', 'pst_wk1_no_city', 
                'pst_wk1_region_count', 'pst_wk1_total_session_time', 
                'pst_wk1_mean_session_time', 'pst_wk1_median_session_time',
                'pst_wk1_no_of_buys', 'pst_wk1_sum_of_buys', 
                'pst_wk2_no_sessions', 'pst_wk2_no_city', 
                'pst_wk2_region_count',  'pst_wk2_total_session_time',
                'pst_wk2_mean_session_time', 'pst_wk2_median_session_time',
                'pst_wk2_no_of_buys', 'pst_wk2_sum_of_buys',
                'pst_wk3_no_sessions', 'pst_wk3_no_city',
                'pst_wk3_region_count', 'pst_wk3_total_session_time',
                'pst_wk3_mean_session_time', 'pst_wk3_median_session_time',
                'pst_wk3_no_of_buys', 'pst_wk3_sum_of_buys',
                'pst_wk4_no_sessions', 'pst_wk4_no_city',
                'pst_wk4_region_count',  'pst_wk4_total_session_time',
                'pst_wk4_mean_session_time', 'pst_wk4_median_session_time',
                'pst_wk4_no_of_buys', 'pst_wk4_sum_of_buys',
                't_total_session_time', 't_no_sessions',
                't_sum_of_buys', 't_no_of_buys']

In [90]:
for c in replace_list:
    df[c].fillna(0, inplace=True)

In [91]:
t = df.isna().sum()
t[t > 0]

last_city               6307974
last_country            6307974
last_region             6307974
last_locale             6307974
user_age                6360718
pst_wk1_last_city       6307974
pst_wk1_last_country    6307974
pst_wk1_last_region     6307974
pst_wk1_last_locale     6307974
pst_wk2_last_city       6307974
pst_wk2_last_country    6307974
pst_wk2_last_region     6307974
pst_wk2_last_locale     6307974
pst_wk3_last_city       6307974
pst_wk3_last_country    6307974
pst_wk3_last_region     6307974
pst_wk3_last_locale     6307974
pst_wk4_last_city       6307974
pst_wk4_last_country    6307974
pst_wk4_last_region     6307974
pst_wk4_last_locale     6307974
dtype: int64

In [92]:
nn_list = ['last_city', 'last_country', 'last_region',
           'last_locale', 'pst_wk1_last_city', 'pst_wk1_last_country',
           'pst_wk1_last_region', 'pst_wk1_last_locale',
           'pst_wk2_last_city', 'pst_wk2_last_country',
           'pst_wk2_last_region', 'pst_wk2_last_locale',
           'pst_wk3_last_city', 'pst_wk3_last_country',
           'pst_wk3_last_region', 'pst_wk3_last_locale',
           'pst_wk4_last_city', 'pst_wk4_last_country',
           'pst_wk4_last_region', 'pst_wk4_last_locale']

In [93]:
for c in nn_list:
    df[c].fillna(-99, inplace=True)

In [94]:
t = df.isna().sum()
t[t > 0]

user_age    6360718
dtype: int64

joined user age again

In [104]:
t = df.isna().sum()
t[t > 0]

user_age    200
dtype: int64

In [105]:
df.sort_values(['anchor_date', 'user_id_hash'], inplace=True)

In [106]:
df.anchor_date.value_counts()

2018-12-08    619535
2018-12-01    619299
2018-11-24    618900
2018-12-15    592208
2018-11-17    556700
2018-12-22    539070
2018-11-10    487804
2018-12-29    477926
2018-11-03    414058
2019-01-05    410978
2019-01-12    345258
2018-10-27    338263
2019-01-19    274438
2018-10-20    265796
2019-01-26    203269
2018-10-13    189794
2019-02-02    131499
2018-10-06    116801
2019-02-09     62826
2018-09-29     48098
2019-02-16     38519
Name: anchor_date, dtype: int64

In [107]:
df.head().T

,0,1,2,3,4
anchor_date,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-09-29 00:00:00
user_id_hash,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,0003f8bda56230a49445880e559b718e94ba37344494a7...,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...
no_sessions,2,1,1,3,1
no_city,1,1,1,1,1
last_city,618,618,448,355,535
last_country,211,211,211,211,224
last_region,117,157,189,85,122
region_count,1,1,1,1,1
last_locale,66,66,66,66,54
total_session_time,403432,0,0,6.17356e+06,0


In [108]:
df.groupby(['anchor_date'])['no_of_buys'].sum()

anchor_date
2018-09-29     6701.0
2018-10-06    20085.0
2018-10-13    25775.0
2018-10-20    29480.0
2018-10-27    29526.0
2018-11-03    31147.0
2018-11-10    30476.0
2018-11-17    30648.0
2018-11-24    30187.0
2018-12-01    18935.0
2018-12-08    12074.0
2018-12-15        0.0
2018-12-22        0.0
2018-12-29        0.0
2019-01-05        0.0
2019-01-12        0.0
2019-01-19        0.0
2019-01-26        0.0
2019-02-02        0.0
2019-02-09        0.0
2019-02-16        0.0
Name: no_of_buys, dtype: float64

In [109]:
df_bkp.shape

(1043065, 17)

In [110]:
df.shape

(7351039, 69)

In [111]:
df = df[df.anchor_date > dt.date(2018, 10, 20)]

In [112]:
df = df[df.anchor_date <= dt.date(2018, 12, 8)]

In [113]:
df.shape

(3654559, 69)

In [192]:
df.anchor_date.value_counts()

2018-12-08    619535
2018-12-01    619299
2018-11-24    618900
2018-11-17    556700
2018-11-10    487804
2018-11-03    414058
2018-10-27    338263
Name: anchor_date, dtype: int64

In [14]:
feature_cols = ['no_sessions', 'no_city',
       'last_city', 'last_country', 'last_region', 'region_count',
       'last_locale', 'total_session_time', 'mean_session_time',
       'median_session_time', 'no_of_buys', 'sum_of_buys',
       'user_age', 'pst_wk1_no_sessions', 'pst_wk1_no_city',
       'pst_wk1_last_city', 'pst_wk1_last_country', 'pst_wk1_last_region',
       'pst_wk1_region_count', 'pst_wk1_last_locale',
       'pst_wk1_total_session_time', 'pst_wk1_mean_session_time',
       'pst_wk1_median_session_time', 'pst_wk1_no_of_buys',
       'pst_wk1_sum_of_buys', 'pst_wk2_no_sessions', 'pst_wk2_no_city',
       'pst_wk2_last_city', 'pst_wk2_last_country', 'pst_wk2_last_region',
       'pst_wk2_region_count', 'pst_wk2_last_locale',
       'pst_wk2_total_session_time', 'pst_wk2_mean_session_time',
       'pst_wk2_median_session_time', 'pst_wk2_no_of_buys',
       'pst_wk2_sum_of_buys', 'pst_wk3_no_sessions', 'pst_wk3_no_city',
       'pst_wk3_last_city', 'pst_wk3_last_country', 'pst_wk3_last_region',
       'pst_wk3_region_count', 'pst_wk3_last_locale',
       'pst_wk3_total_session_time', 'pst_wk3_mean_session_time',
       'pst_wk3_median_session_time', 'pst_wk3_no_of_buys',
       'pst_wk3_sum_of_buys', 'pst_wk4_no_sessions', 'pst_wk4_no_city',
       'pst_wk4_last_city', 'pst_wk4_last_country', 'pst_wk4_last_region',
       'pst_wk4_region_count', 'pst_wk4_last_locale',
       'pst_wk4_total_session_time', 'pst_wk4_mean_session_time',
       'pst_wk4_median_session_time', 'pst_wk4_no_of_buys',
       'pst_wk4_sum_of_buys', 't_total_session_time', 't_no_sessions',
       't_sum_of_buys', 't_no_of_buys']

In [10]:
feature_cols += [i for i in df.columns if i.startswith('event_')]

In [11]:
feature_cols += [i for i in df.columns if i.startswith('atr_')]

In [116]:
df.user_id_hash.unique().shape

(619535,)

for week 08 to 14 you don't know y1 and y2  
for week 01 to 08 you don't know y2 but know y1  
for week 24 to 01 you know both y1 and y2  

In [117]:
df.groupby(['anchor_date'])['user_id_hash'].agg('nunique')

anchor_date
2018-10-27    338263
2018-11-03    414058
2018-11-10    487804
2018-11-17    556700
2018-11-24    618900
2018-12-01    619299
2018-12-08    619535
Name: user_id_hash, dtype: int64

In [118]:
df.shape

(3654559, 69)

In [9]:
df.shape

(3654559, 337)

In [10]:
t = df.isna().sum()
t[t > 0]

Series([], dtype: int64)

## modelling

In [15]:
def print_feature_importance(model):
    feature_importance = dict(zip(feature_cols, model.feature_importances_))
    for name, importance in sorted(feature_importance.items(), key=lambda x: x[1],\
                                   reverse=True):
        print(f"{name:<30}: {importance:>6.2%}")

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [30]:
train = df[df.anchor_date < dt.date(2018, 12, 1)]
test = df[(df.anchor_date == dt.date(2018, 12, 1))]

x_train = train[feature_cols].values
y1_train = train[['y1']]

x_test = test[feature_cols].values
y1_test = test[['y1']]

In [17]:
y1_train.shape

(2415725, 1)

In [18]:
y1_train.sum()

y1    33510.0
dtype: float64

In [196]:
(2415725 - 33510.0) / 33510.0

71.089674723963

### without events and attributes  - train test y1

In [70]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'num_boost_round' : 500,
    'n_estimators': 300,
    'n_jobs': -1,
    'scale_pos_weight': 70.0
    }

In [64]:
xg_model1 = XGBClassifier(**params)

In [65]:
xg_model1.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=15.0, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, num_boost_round=500,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=70.0, seed=None, silent=True,
       subsample=1)

In [66]:
accuracy_score(y1_train, xg_model1.predict(x_train))

0.9803413054052096

In [67]:
accuracy_score(y1_test, xg_model1.predict(x_test))

0.9862166740136832

In [68]:
roc_auc_score(y1_train, xg_model1.predict(x_train))

0.9880758187782617

In [69]:
roc_auc_score(y1_test, xg_model1.predict(x_test))

0.9582613023737487

In [132]:
print_feature_importance(xg_model1)

total_session_time            :  7.13%
t_sum_of_buys                 :  4.62%
user_age                      :  4.46%
last_city                     :  4.37%
t_total_session_time          :  4.16%
no_sessions                   :  3.97%
pst_wk1_last_city             :  3.95%
last_region                   :  3.68%
pst_wk1_total_session_time    :  3.64%
mean_session_time             :  3.24%
pst_wk1_last_region           :  2.97%
median_session_time           :  2.88%
pst_wk1_mean_session_time     :  2.82%
no_of_buys                    :  2.39%
pst_wk1_no_sessions           :  2.35%
pst_wk1_median_session_time   :  2.34%
t_no_sessions                 :  2.29%
last_locale                   :  2.00%
t_no_of_buys                  :  1.92%
sum_of_buys                   :  1.88%
pst_wk2_last_city             :  1.86%
pst_wk2_total_session_time    :  1.84%
pst_wk2_no_sessions           :  1.76%
last_country                  :  1.69%
pst_wk1_last_locale           :  1.38%
pst_wk3_last_city        

In [47]:
imp_cols = ['total_session_time', 'atr_mean19', 'last_city',
            'pst_wk1_last_city', 'event_count45', 'atr_mean13',
            'user_age', 'event_sum7', 'last_region', 'atr_mean5',
            'pst_wk1_last_region', 't_total_session_time', 'atr_mean16',
            'atr_mean20', 'atr_mean7', 'atr_last19', 'event_mean7',
            'atr_max13', 'pst_wk1_total_session_time', 'atr_last13',
            'atr_mean8', 'mean_session_time', 'atr_mean1', 't_sum_of_buys',
            'event_count1', 'event_mean5', 'pst_wk1_no_sessions',
            'pst_wk1_mean_session_time', 'atr_last1', 'pst_wk2_last_city',
            'median_session_time', 'atr_last5', 'atr_mean66',
            'pst_wk1_last_country', 't_no_sessions', 'event_count98',
            'event_count0', 'atr_mean67', 'atr_mean4', 'atr_count67',
            'last_locale', 'atr_max1', 'last_country', 'pst_wk1_no_city',
            'pst_wk1_median_session_time', 'atr_max19', 'atr_mean0',
            'pst_wk1_last_locale', 'pst_wk2_no_sessions', 'event_count14', 
            't_no_of_buys', 'atr_mean18', 'atr_last7', 'no_of_buys',
            'atr_mean75', 'atr_mean15', 'atr_mean17', 'atr_max7',
            'no_city', 'atr_max16', 'no_sessions', 'atr_last18',
            'pst_wk2_last_region', 'pst_wk3_last_city', 'atr_mean9',
            'atr_mean6', 'event_count7', 'pst_wk2_total_session_time',
            'event_count40', 'event_count42', 'event_mode7',
            'pst_wk3_last_region', 'pst_wk3_total_session_time',
            'atr_mean3', 'atr_max5', 'atr_mean14', 'atr_last20',
            'pst_wk2_mean_session_time', 'event_count4', 'event_count5',
            'atr_last16', 'atr_max15', 'pst_wk2_median_session_time',
            'event_count41', 'atr_max18', 'atr_max66', 
            'pst_wk1_no_of_buys', 'event_count6', 'event_count3',
            'pst_wk3_no_sessions', 'atr_max6', 'pst_wk2_sum_of_buys', 
            'event_sum5', 'atr_last75', 'atr_mean21', 'atr_count0', 
            'sum_of_buys', 'pst_wk3_last_locale', 'pst_wk4_last_region',
            'atr_last6', 'pst_wk1_sum_of_buys', 'atr_last66', 'atr_max9',
            'pst_wk2_last_locale', 'atr_count7', 'pst_wk2_last_country',
            'pst_wk4_last_city', 'pst_wk4_median_session_time',
            'atr_last4', 'pst_wk4_total_session_time', 'event_count97',
            'atr_max4', 'atr_max75', 'pst_wk3_last_country', 'atr_max8',
            'atr_mean11', 'pst_wk2_no_city', 'pst_wk1_region_count',
            'pst_wk3_no_city', 'pst_wk4_mean_session_time',
            'pst_wk4_sum_of_buys', 'pst_wk2_region_count', 
            'pst_wk3_mean_session_time', 'pst_wk3_sum_of_buys', 
            'atr_count66', 'atr_last9', 'region_count', 'atr_last15', 
            'pst_wk4_no_sessions', 'atr_last8', 'event_count44', 
            'pst_wk4_last_locale', 'atr_mean96', 'atr_last3', 
            'pst_wk3_median_session_time', 'atr_max14']

In [48]:
imp_cols = listimp_colst(imp_cols) - set(features))

NameError: name 'features' is not defined

### with events and attributes

In [20]:
xg_model1 = XGBClassifier(**params)

In [21]:
xg_model1.fit(x_train, y1_train)

/home/jupyter/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=10.0, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, num_boost_round=700,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=70.0, seed=None, silent=True,
       subsample=1)

In [22]:
accuracy_score(y1_train, xg_model1.predict(x_train))

0.9846567800556769

In [23]:
accuracy_score(y1_test, xg_model1.predict(x_test))

0.9891151124093531

In [24]:
roc_auc_score(y1_train, xg_model1.predict(x_train))

0.9919262548732

In [25]:
roc_auc_score(y1_test, xg_model1.predict(x_test))

0.9574480169884488

In [26]:
print_feature_importance(xg_model1)

total_session_time            :  2.68%
atr_mean19                    :  2.49%
last_city                     :  2.46%
pst_wk1_last_city             :  2.46%
event_count45                 :  2.43%
atr_mean13                    :  2.41%
user_age                      :  2.27%
event_sum7                    :  1.95%
last_region                   :  1.86%
atr_mean5                     :  1.78%
pst_wk1_last_region           :  1.74%
t_total_session_time          :  1.72%
atr_mean16                    :  1.72%
atr_mean20                    :  1.64%
atr_mean7                     :  1.62%
atr_last19                    :  1.59%
event_mean7                   :  1.58%
atr_max13                     :  1.53%
pst_wk1_total_session_time    :  1.49%
atr_last13                    :  1.42%
atr_mean8                     :  1.41%
mean_session_time             :  1.41%
atr_mean1                     :  1.38%
t_sum_of_buys                 :  1.25%
event_count1                  :  1.20%
event_mean5              

----

In [80]:
train = df[df.anchor_date <= dt.date(2018, 11, 24)]

x_train = train[feature_cols].values
y2_train = train[['y2']]

In [81]:
y2_train.sum()

y2    25427.0
dtype: float64

In [82]:
y2_train.shape

(1929779, 1)

In [83]:
(1929779.0 - 25427.0) / 25427.0

74.89487552601565

In [84]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'num_boost_round' : 700,
    'n_estimators': 300,
    'n_jobs': -1,
    'scale_pos_weight': 74.0
    }

In [85]:
xg_model2 = XGBClassifier(**params)

In [86]:
xg_model2.fit(x_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=10.0, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, num_boost_round=700,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=74.0, seed=None, silent=True,
       subsample=1)

In [87]:
accuracy_score(y2_train, xg_model2.predict(x_train))

0.922508743229147

In [88]:
roc_auc_score(y2_train, xg_model2.predict(x_train))

0.9453127819743455

### y1 total train

In [19]:
train = df[df.anchor_date <= dt.date(2018, 12, 1)]

x_train = train[feature_cols].values
y1_train = train[['y1']]

In [20]:
y1_train.shape

(3035024, 1)

In [21]:
y1_train.sum()

y1    36796.0
dtype: float64

In [22]:
(3035024 - 36796.0) / 36796.0

81.4824437438852

In [23]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'num_boost_round' : 500,
    'n_estimators': 300,
    'n_jobs': -1,
    'scale_pos_weight': 75.0
    }

In [24]:
xg_model1 = XGBClassifier(**params)

In [25]:
xg_model1.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=10.0, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, num_boost_round=500,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=75.0, seed=None, silent=True,
       subsample=1)

In [26]:
accuracy_score(y1_train, xg_model1.predict(x_train))

0.9806232174770282

In [27]:
roc_auc_score(y1_train, xg_model1.predict(x_train))

0.9878170718600007

In [28]:
import pickle

In [29]:
pickle.dump(xg_model1, open("xg_model1.dat", "wb"))

In [30]:
# xg_model1 = pickle.load(open("xg_model1.dat", "rb"))

In [32]:
del train
del y1_train
del x_train

In [33]:
gc.collect()

1897

### y2 total train

In [34]:
train2 = df[df.anchor_date <= dt.date(2018, 11, 24)]

x_train = train2[feature_cols].values
y2_train = train2[['y2']]

In [35]:
y2_train.shape

(2415725, 1)

In [36]:
y2_train.sum()

y2    50456.0
dtype: float64

In [85]:
(2415725 - 50456.0) / 50456.0

46.87785397177739

In [47]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'num_boost_round' : 500,
    'n_estimators': 300,
    'n_jobs': -1,
    'scale_pos_weight': 45.0
    }

In [48]:
xg_model2 = XGBClassifier(**params)

In [49]:
xg_model2.fit(x_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=10.0, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, num_boost_round=500,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=45.0, seed=None, silent=True,
       subsample=1)

In [50]:
accuracy_score(y2_train, xg_model2.predict(x_train))

0.9797187179832142

In [51]:
roc_auc_score(y2_train, xg_model2.predict(x_train))

0.978557959561948

In [61]:
pickle.dump(xg_model2, open("xg_model2.dat", "wb"))

### Random forest

In [164]:
from sklearn.ensemble import RandomForestClassifier

In [178]:
rf1 = RandomForestClassifier(n_estimators=300, min_samples_leaf=50,
                             n_jobs=-1, class_weight='balanced')

In [179]:
rf1.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=50,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [180]:
accuracy_score(y1_train, rf1.predict(x_train))

0.9673061008540356

In [181]:
accuracy_score(y1_test, rf1.predict(x_test))

0.9793152098503443

In [182]:
roc_auc_score(y1_train, rf1.predict(x_train))

0.9710510802009252

In [183]:
roc_auc_score(y1_test, rf1.predict(x_test))

0.9377755248413372

In [171]:
print_feature_importance(rf1)

atr_mean16                    :  4.13%
atr_max16                     :  3.86%
atr_max66                     :  3.83%
atr_max5                      :  3.63%
atr_mean13                    :  3.62%
atr_last66                    :  3.33%
atr_mean66                    :  3.29%
atr_last16                    :  3.17%
atr_max6                      :  3.16%
atr_max13                     :  3.10%
atr_mean5                     :  2.60%
atr_mean6                     :  2.52%
atr_max9                      :  2.45%
atr_last9                     :  1.74%
total_session_time            :  1.69%
atr_mean9                     :  1.64%
atr_last13                    :  1.59%
atr_max7                      :  1.56%
atr_last6                     :  1.48%
atr_mean10                    :  1.39%
atr_mean7                     :  1.39%
event_mean5                   :  1.35%
atr_count10                   :  1.24%
atr_last10                    :  1.19%
atr_max10                     :  1.16%
atr_last7                

rf is over fitting

# Whole data

In [260]:
df = df[df.anchor_date >= dt.date(2018, 10, 1)]

In [262]:
df.anchor_date.max()

Timestamp('2018-12-14 00:00:00')

In [263]:
df = df[df.anchor_date <= dt.datetime(2018, 12, 8)]

In [264]:
df.shape

(6652857, 17)

In [265]:
df.reset_index(inplace=True, drop=True)

In [266]:
df.user_id_hash = df.user_id_hash.cat.codes
df.last_city = df.last_city.cat.codes
df.last_country = df.last_country.cat.codes
df.last_region = df.last_region.cat.codes
df.last_locale = df.last_locale.cat.codes

In [267]:
df['y1'] = df['y1'].astype(np.int)
df['y2'] = df['y2'].astype(np.int)

In [268]:
df.isna().sum()

anchor_date            0
user_id_hash           0
no_sessions            0
no_city                0
last_city              0
last_country           0
last_region            0
region_count           0
last_locale            0
total_session_time     0
mean_session_time      0
median_session_time    0
no_of_buys             0
sum_of_buys            0
y1                     0
y2                     0
user_age               0
dtype: int64

In [269]:
train = df[df.anchor_date < dt.date(2018, 12, 1)]

In [270]:
test = df[df.anchor_date == dt.date(2018, 12, 1)]

In [271]:
test.shape

(49600, 17)

In [274]:
x_train = train[feature_cols].values
y1_train = train[['y1']]
y2_train = train[['y2']]

x_test = test[feature_cols].values
y1_test = test[['y1']]
y2_test = test[['y2']]

In [275]:
feature_cols = ['no_sessions', 'no_city', 'last_city',
                'last_country', 'last_region', 'region_count', 'last_locale',
                'total_session_time', 'mean_session_time', 'median_session_time',
                'no_of_buys', 'sum_of_buys', 'user_age']

In [276]:
rf1 = RandomForestClassifier(n_estimators=300, min_samples_leaf=10, n_jobs=-1)

In [277]:
rf1.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [218]:
accuracy_score(y1_train, rf1.predict(x_train))

0.9751309585713738

In [278]:
accuracy_score(y1_test, rf1.predict(x_test))

0.9546774193548387

In [220]:
roc_auc_score(y1_train, rf1.predict(x_train))

0.687500406308949

In [279]:
roc_auc_score(y1_test, rf1.predict(x_test))

0.662303911233751

In [280]:
print_feature_importance(rf1)

sum_of_buys                   : 27.28%
no_of_buys                    : 24.21%
total_session_time            :  8.61%
mean_session_time             :  7.22%
median_session_time           :  7.00%
user_age                      :  6.39%
no_sessions                   :  5.52%
last_region                   :  4.35%
last_city                     :  3.95%
last_locale                   :  1.89%
last_country                  :  1.65%
no_city                       :  1.27%
region_count                  :  0.66%


# With Events and subset

In [135]:
events = pd.read_pickle('../../events_roll_agg.pkl')

In [136]:
attrib = pd.read_pickle('../../attributes_agg2.pkl')

In [137]:
x

datetime.date(2018, 12, 8)

In [138]:
# df = df[df.anchor_date.isin([x - dt.timedelta(days=7*i) for i in range(10)])]

In [139]:
df.shape

(3654559, 69)

In [134]:
df.to_pickle('../../datav1.pkl')

In [140]:
events.shape

(56566389, 7)

In [141]:
attrib.shape

(24242895, 7)

In [142]:
events = events[events.anchor_date.isin([x - dt.timedelta(days=7*i) 
                                         for i in range(11)])]

In [143]:
events.shape

(8019130, 7)

In [144]:
attrib.anchor_date.isin([x - dt.timedelta(days=7*i)
                         for i in range(11)]).sum()

24242895

In [145]:
attrib = attrib[attrib.anchor_date.isin([x - dt.timedelta(days=7*i)
                                         for i in range(11)])]

In [146]:
attrib.shape

(24242895, 7)

In [147]:
events = events[events.event != '8']

In [148]:
events.head()

,anchor_date,event,user_id_hash,event_count,event_sum,event_mean,event_mode
0,2018-09-29,0,001649233906501efe5899fc693500a233220cfc02be0d...,1,0.0,0.0,0.0
1,2018-09-29,0,001b256fc745aff7b582d02dd1e7d9209a7b6c486b4cf2...,1,0.0,0.0,0.0
2,2018-09-29,0,001f75076b085ab0cb6b6b39d25e99f2aba1c6eb64b134...,1,0.0,0.0,0.0
3,2018-09-29,0,0023f69f80e87162b3f1daae9a5795a1b1820cc8fc4fa9...,1,0.0,0.0,0.0
4,2018-09-29,0,0025884539ef9c52b2c55414fc78b178bbd6ffb4f7e6ba...,1,0.0,0.0,0.0


In [149]:
attrib.head()

,anchor_date,attribute,user_id_hash,atr_count,atr_mean,atr_last,atr_max
0,2018-09-29,0,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
1,2018-09-29,0,0003f8bda56230a49445880e559b718e94ba37344494a7...,1,0.0,0.0,0.0
2,2018-09-29,0,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,0.0,0.0,0.0
3,2018-09-29,0,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,3,0.0,0.0,0.0
4,2018-09-29,0,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...,1,0.0,0.0,0.0


In [150]:
events.dtypes

anchor_date     datetime64[ns]
event                 category
user_id_hash          category
event_count              int64
event_sum              float64
event_mean             float64
event_mode             float64
dtype: object

In [151]:
attrib.anchor_date = attrib.anchor_date.astype('datetime64')

In [152]:
attrib.dtypes

anchor_date     datetime64[ns]
attribute             category
user_id_hash          category
atr_count                int64
atr_mean               float64
atr_last               float64
atr_max                float64
dtype: object

In [153]:
events.sort_values(['anchor_date', 'user_id_hash'], inplace=True)

In [154]:
events.reset_index(drop=True, inplace=True)

In [155]:
events.head()

,anchor_date,event,user_id_hash,event_count,event_sum,event_mean,event_mode
0,2018-09-29,14,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,3,0.0,0.0,0.0
1,2018-09-29,44,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,1,0.0,0.0,0.0
2,2018-09-29,45,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,12,0.0,0.0,0.0
3,2018-09-29,5,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,1,0.0,0.0,0.0
4,2018-09-29,7,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,1,12.0,12.0,12.0


In [156]:
attrib.sort_values(['anchor_date', 'user_id_hash'], inplace=True)

In [157]:
attrib.reset_index(drop=True, inplace=True)

In [158]:
attrib.head()

,anchor_date,attribute,user_id_hash,atr_count,atr_mean,atr_last,atr_max
0,2018-09-29,0,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
1,2018-09-29,1,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
2,2018-09-29,3,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
3,2018-09-29,4,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
4,2018-09-29,5,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,1.0,1.0,1.0


In [159]:
events.set_index(['anchor_date', 'user_id_hash', 'event'], inplace=True)

In [160]:
attrib.set_index(['anchor_date', 'user_id_hash', 'attribute'], inplace=True)

In [161]:
events_p = events.unstack()

In [162]:
attrib_p = attrib.unstack()

In [163]:
events_p.head()

event_count  \
event                                                                   14   
anchor_date user_id_hash                                                     
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...         3.0   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...         3.0   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...         4.0   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...         3.0   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...         6.0   

                                                                           \
event                                                            44    45   
anchor_date user_id_hash                                                    
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  1.0  12.0   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...  1.0   1.0   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  1.0  29.0   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  1.0  72.0   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...  1.0  14.0   

                                                                               \
event                                                             5    7   97   
anchor_date user_id_hash                                                        
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  1.0  1.0  3.0   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...  1.0  NaN  4.0   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  3.0  1.0  2.0   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  5.0  3.0  4.0   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...  1.0  NaN  3.0   

                                                                               \
event                                                             1    6    4   
anchor_date user_id_hash                                                        
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  NaN  NaN  NaN   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...  NaN  NaN  NaN   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  1.0  1.0  NaN   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  2.0  4.0  3.0   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...  NaN  NaN  NaN   

                                                                    ...  \
event                                                            40 ...   
anchor_date user_id_hash                                            ...   
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  NaN ...   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...  NaN ...   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  NaN ...   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  2.0 ...   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...  NaN ...   

                                                               event_mode      \
event                                                                  52  55   
anchor_date user_id_hash                                                        
2018-09-29  00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...        NaN NaN   
            0003f8bda56230a49445880e559b718e94ba37344494a7f...        NaN NaN   
            000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...        NaN NaN   
            0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...        NaN NaN   
            0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae1...        NaN NaN   

                                                                            \
event                                                           60  57  64   
anchor_date user_id_hash                                                     
2018-09-29  00026e5050a70ef12d421f75c6a

In [164]:
for c in events_p.columns[events_p.isna().sum() > 0].values:
    events_p[c].fillna(-99, inplace=True)

In [165]:
attrib_p.fillna(-99, inplace=True)

In [166]:
events_p.columns = [i + str(j) for  i, j  in zip (events_p.columns.get_level_values(0),
                                                  events_p.columns.get_level_values(1))]

In [167]:
attrib_p.columns = [i + str(j) for  i, j  in zip (attrib_p.columns.get_level_values(0),
                                                  attrib_p.columns.get_level_values(1))]

In [168]:
events_p.reset_index(inplace=True)

In [169]:
attrib_p.reset_index(inplace=True)

In [170]:
events_p.head()

,anchor_date,user_id_hash,event_count14,event_count44,event_count45,event_count5,event_count7,event_count97,event_count1,event_count6,...,event_mode52,event_mode55,event_mode60,event_mode57,event_mode64,event_mode32,event_mode61,event_mode0/,event_mode1/,event_mode62
0,2018-09-29,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,3.0,1.0,12.0,1.0,1.0,3.0,-99.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,2018-09-29,0003f8bda56230a49445880e559b718e94ba37344494a7...,3.0,1.0,1.0,1.0,-99.0,4.0,-99.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,2018-09-29,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,4.0,1.0,29.0,3.0,1.0,2.0,1.0,1.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,2018-09-29,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,3.0,1.0,72.0,5.0,3.0,4.0,2.0,4.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,2018-09-29,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...,6.0,1.0,14.0,1.0,-99.0,3.0,-99.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [171]:
attrib_p.head()

,anchor_date,user_id_hash,atr_count0,atr_count1,atr_count3,atr_count4,atr_count5,atr_count6,atr_count8,atr_count9,...,atr_max21,atr_max66,atr_max75,atr_max76,atr_max77,atr_max78,atr_max7,atr_max10,atr_max67,atr_max96
0,2018-09-29,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,1.0,0.0,0.0,0.0,1.0,-99.0,-99.0,-99.0,-99.0
1,2018-09-29,0003f8bda56230a49445880e559b718e94ba37344494a7...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,0.0,0.0,0.0,1.0,-99.0,-99.0,-99.0,-99.0
2,2018-09-29,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,-99.0,-99.0
3,2018-09-29,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,0.0,5.0,0.0,0.0,0.0,1.0,4.0,1.0,-99.0,-99.0
4,2018-09-29,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,0.0,0.0,0.0,1.0,-99.0,-99.0,-99.0,-99.0


In [172]:
events_p[['user_id_hash', 'anchor_date']].dtypes

user_id_hash          category
anchor_date     datetime64[ns]
dtype: object

In [173]:
df[['user_id_hash', 'anchor_date']].dtypes

user_id_hash            object
anchor_date     datetime64[ns]
dtype: object

In [174]:
df.shape

(3654559, 69)

In [175]:
df = df.merge(events_p, on=['user_id_hash', 'anchor_date'], how='left')

In [176]:
df = df.merge(attrib_p, on=['user_id_hash', 'anchor_date'], how='left')

In [189]:
df[['user_id_hash', 'anchor_date']].head()

,user_id_hash,anchor_date
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,2018-10-27
1,000062e9be78f3da274fec338e78f89d12000e781967f2...,2018-10-27
2,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,2018-10-27
3,0000c5de50d243cc39f6ac4268d7511705585ca3f2a20f...,2018-10-27
4,0000c91fb9908357cc11974d0fc67a260ddb46508055f2...,2018-10-27


In [177]:
df.shape

(3654559, 337)

In [178]:
feature_cols += [i for i in df.columns if i.startswith('event_')]

In [179]:
feature_cols += [i for i in df.columns if i.startswith('atr_')]

In [180]:
np.array(feature_cols)

array(['no_sessions', 'no_city', 'last_city', 'last_country',
       'last_region', 'region_count', 'last_locale', 'total_session_time',
       'mean_session_time', 'median_session_time', 'no_of_buys',
       'sum_of_buys', 'user_age', 'pst_wk1_no_sessions',
       'pst_wk1_no_city', 'pst_wk1_last_city', 'pst_wk1_last_country',
       'pst_wk1_last_region', 'pst_wk1_region_count',
       'pst_wk1_last_locale', 'pst_wk1_total_session_time',
       'pst_wk1_mean_session_time', 'pst_wk1_median_session_time',
       'pst_wk1_no_of_buys', 'pst_wk1_sum_of_buys', 'pst_wk2_no_sessions',
       'pst_wk2_no_city', 'pst_wk2_last_city', 'pst_wk2_last_country',
       'pst_wk2_last_region', 'pst_wk2_region_count',
       'pst_wk2_last_locale', 'pst_wk2_total_session_time',
       'pst_wk2_mean_session_time', 'pst_wk2_median_session_time',
       'pst_wk2_no_of_buys', 'pst_wk2_sum_of_buys', 'pst_wk3_no_sessions',
       'pst_wk3_no_city', 'pst_wk3_last_city', 'pst_wk3_last_country',
       'pst_wk3_

In [181]:
t = df.isna().sum()
t[t > 0]

user_age             107
event_count14    2877411
event_count44    2877411
event_count45    2877411
event_count5     2877411
event_count7     2877411
event_count97    2877411
event_count1     2877411
event_count6     2877411
event_count4     2877411
event_count40    2877411
event_count11    2877411
event_count9     2877411
event_count3     2877411
event_count41    2877411
event_count42    2877411
event_count0     2877411
event_count43    2877411
event_count10    2877411
event_count47    2877411
event_count51    2877411
event_count58    2877411
event_count98    2877411
event_count50    2877411
event_count54    2877411
event_count59    2877411
event_count49    2877411
event_count56    2877411
event_count99    2877411
event_count63    2877411
event_count48    2877411
event_count52    2877411
event_count55    2877411
event_count60    2877411
event_count57    2877411
event_count64    2877411
event_count32    2877411
event_count61    2877411
event_count0/    2877411
event_count1/    2877411


In [185]:
t[t > 0].index.values

array(['user_age', 'event_count14', 'event_count44', 'event_count45',
       'event_count5', 'event_count7', 'event_count97', 'event_count1',
       'event_count6', 'event_count4', 'event_count40', 'event_count11',
       'event_count9', 'event_count3', 'event_count41', 'event_count42',
       'event_count0', 'event_count43', 'event_count10', 'event_count47',
       'event_count51', 'event_count58', 'event_count98', 'event_count50',
       'event_count54', 'event_count59', 'event_count49', 'event_count56',
       'event_count99', 'event_count63', 'event_count48', 'event_count52',
       'event_count55', 'event_count60', 'event_count57', 'event_count64',
       'event_count32', 'event_count61', 'event_count0/', 'event_count1/',
       'event_count62', 'event_sum14', 'event_sum44', 'event_sum45',
       'event_sum5', 'event_sum7', 'event_sum97', 'event_sum1',
       'event_sum6', 'event_sum4', 'event_sum40', 'event_sum11',
       'event_sum9', 'event_sum3', 'event_sum41', 'event_sum42',


In [186]:
df.fillna(-99, inplace=True)

In [187]:
t = df.isna().sum()
t[t > 0]

Series([], dtype: int64)

In [188]:
df.to_pickle('../../datav2.pkl')

In [8]:
df = pd.read_pickle('../../datav2.pkl')

In [60]:
df.user_id_hash = df.user_id_hash.astype('category')
df.last_city = df.last_city.astype('category')
df.last_country = df.last_country.astype('category')
df.last_region = df.last_region.astype('category')
df.last_locale = df.last_locale.astype('category')

df.last_city = df.last_city.cat.codes
df.last_country = df.last_country.cat.codes
df.last_region = df.last_region.cat.codes
df.last_locale = df.last_locale.cat.codes

df['y1'] = df['y1'].astype(np.int)
df['y2'] = df['y2'].astype(np.int)

In [61]:
df.isna().sum()

anchor_date                0
user_id_hash               0
no_sessions                0
no_city                    0
last_city                  0
last_country               0
last_region                0
region_count               0
last_locale                0
total_session_time         0
mean_session_time          0
median_session_time        0
no_of_buys                 0
sum_of_buys                0
y1                         0
y2                         0
user_age                   0
index_x                31045
event_count1           31045
event_count14          31045
event_count3           31045
event_count4           31045
event_count40          31045
event_count41          31045
event_count42          31045
event_count45          31045
event_count5           31045
event_count6           31045
event_count7           31045
event_count97          31045
                       ...  
atr_last78                 0
atr_last67                 0
atr_last96                 0
atr_max0      

In [62]:
for c in df.columns[df.isna().sum() > 0].values:
    df[c].fillna(-50, inplace=True)

In [10]:
from warnings import filterwarnings

In [11]:
filterwarnings('ignore')

In [103]:
df.to_pickle('../thedf')

In [9]:
df = pd.read_pickle('../thedf')

In [34]:
train = df[df.anchor_date < dt.date(2018, 12, 10)]
# test = df[(df.anchor_date >= dt.date(2018, 12, 1)) &
#           (df.anchor_date < dt.date(2018, 12, 8))]

In [35]:
x_train = train[feature_cols].values
y1_train = train[['y1']]
y2_train = train[['y2']]

In [90]:
# x_test = test[feature_cols].values
# y1_test = test[['y1']]
# y2_test = test[['y2']]

In [37]:
rf1 = RandomForestClassifier(n_estimators=400, min_samples_leaf=10, n_jobs=-1)

In [38]:
rf1.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
accuracy_score(y1_train, rf1.predict(x_train))

0.9787527698638809

In [72]:
# accuracy_score(y1_test, rf1.predict(x_test))

In [40]:
roc_auc_score(y1_train, rf1.predict(x_train))

0.7183990323437283

In [329]:
# roc_auc_score(y1_test, rf1.predict(x_test))

0.6127301313633443

In [41]:
print_feature_importance(rf1)

sum_of_buys                   :  5.45%
no_of_buys                    :  4.57%
event_sum7                    :  3.85%
atr_mean14                    :  3.53%
atr_max1                      :  3.52%
event_count7                  :  3.40%
atr_mean1                     :  3.20%
atr_max15                     :  3.01%
atr_mean15                    :  2.75%
atr_last1                     :  2.27%
atr_mean13                    :  2.23%
atr_max14                     :  2.08%
atr_last13                    :  2.01%
atr_max13                     :  1.88%
atr_last14                    :  1.82%
user_age                      :  1.78%
atr_last15                    :  1.75%
event_count45                 :  1.40%
atr_last16                    :  0.91%
atr_mean16                    :  0.90%
atr_mean19                    :  0.83%
atr_mean7                     :  0.82%
event_count4                  :  0.78%
atr_max16                     :  0.77%
mean_session_time             :  0.77%
median_session_time      

In [42]:
xg1 = XGBClassifier(max_depth=5, learning_rate=0.01,
                          n_estimators=300, n_jobs=-1)

In [43]:
xg1.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [44]:
accuracy_score(y1_train, xg1.predict(x_train))

0.9692866941015089

In [72]:
# accuracy_score(y1_test, rf1.predict(x_test))

In [45]:
roc_auc_score(y1_train, xg1.predict(x_train))

0.630694651904953

In [46]:
print_feature_importance(xg1)

user_age                      : 14.30%
event_count0                  :  8.44%
event_count97                 :  4.83%
last_country                  :  4.11%
atr_mean14                    :  3.25%
last_locale                   :  3.15%
event_count55                 :  3.12%
event_sum7                    :  2.67%
total_session_time            :  2.39%
event_count41                 :  2.35%
atr_mean15                    :  2.09%
atr_count66                   :  2.07%
atr_max15                     :  1.90%
event_count42                 :  1.78%
event_count7                  :  1.78%
event_count98                 :  1.72%
sum_of_buys                   :  1.61%
atr_mean11                    :  1.43%
event_count64                 :  1.40%
atr_mean16                    :  1.40%
atr_mean9                     :  1.34%
atr_last67                    :  1.26%
event_count45                 :  1.24%
atr_last7                     :  1.21%
atr_max16                     :  1.21%
atr_mean66               

### y2

In [47]:
train2 = df[df.anchor_date < dt.date(2018, 12, 3)]
x2_train = train2[feature_cols].values
y2_train = train2[['y2']]

In [48]:
rf2 = RandomForestClassifier(n_estimators=400, min_samples_leaf=10, n_jobs=-1)

In [49]:
rf2.fit(x2_train, y2_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
accuracy_score(y2_train, rf2.predict(x2_train))

0.9750677226598975

In [51]:
roc_auc_score(y2_train, rf2.predict(x2_train))

0.7442983094954112

In [52]:
print_feature_importance(rf2)

sum_of_buys                   :  4.43%
no_of_buys                    :  4.41%
atr_max1                      :  4.01%
atr_mean14                    :  3.86%
atr_last14                    :  3.29%
event_sum7                    :  3.22%
atr_max15                     :  3.17%
atr_mean15                    :  3.15%
atr_mean1                     :  2.93%
atr_max13                     :  2.84%
atr_last13                    :  2.79%
atr_last15                    :  2.71%
atr_max14                     :  2.56%
event_count7                  :  2.23%
atr_mean13                    :  2.13%
atr_last1                     :  1.87%
user_age                      :  1.44%
atr_max16                     :  1.24%
event_count45                 :  1.23%
atr_mean16                    :  1.17%
atr_last16                    :  1.10%
atr_mean3                     :  0.99%
total_session_time            :  0.93%
event_mean7                   :  0.81%
mean_session_time             :  0.77%
median_session_time      

# submission

In [52]:
submission = pd.read_csv('../../sample_submission_2.csv')
req_uids = set(submission.user_id_hash)
dates = df.anchor_date.dt.date.unique()
dates.sort()

In [53]:
req_uids = set(submission.user_id_hash)
run_ids = req_uids
predict_df = pd.DataFrame(columns=['user_id_hash',
                                   'user_purchase_binary_7_days',
                                   'user_purchase_binary_14_days'])

In [54]:
sorted(dates, reverse=True)[0]

datetime.date(2018, 12, 8)

In [55]:
df.anchor_date.unique()

array(['2018-10-27T00:00:00.000000000', '2018-11-03T00:00:00.000000000',
       '2018-11-10T00:00:00.000000000', '2018-11-17T00:00:00.000000000',
       '2018-11-24T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [56]:
dts = dt.date(2018, 12, 8)

In [57]:
dates

array([datetime.date(2018, 10, 27), datetime.date(2018, 11, 3),
       datetime.date(2018, 11, 10), datetime.date(2018, 11, 17),
       datetime.date(2018, 11, 24), datetime.date(2018, 12, 1),
       datetime.date(2018, 12, 8)], dtype=object)

In [58]:
t = df.loc[(df.user_id_hash.isin(run_ids)) & \
           (df.anchor_date == dts), feature_cols].values

In [59]:
cur_uids = df.loc[(df.user_id_hash.isin(run_ids)) & \
                  (df.anchor_date == dts), ['user_id_hash']].values

In [60]:
pp1 = xg_model1.predict_proba(t)
pp2 = xg_model2.predict_proba(t)

In [62]:
temp = pd.DataFrame(data={'user_id_hash': cur_uids.reshape(-1),
                          'user_purchase_binary_7_days': pp1[:,1],
                          'user_purchase_binary_14_days': pp2[:,1]})

In [63]:
predict_df = pd.concat([predict_df, temp])

In [64]:
run_ids = run_ids - set(cur_uids.reshape(-1))

In [65]:
len(run_ids)

994

In [66]:
predict_df.shape

(311574, 3)

In [67]:
left_out = list(set(submission.user_id_hash.unique()) -\
                set(predict_df.user_id_hash.unique()))

In [56]:
for dts in reversed(dates[-3:]):
    t = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.anchor_date == dts), feature_cols].values
    cur_uids = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.anchor_date == dts), ['user_id_hash']].values

    pp1 = xg_model1.predict_proba(t)
    pp2 = xg_model2.predict_proba(t)

    print(f'{dts}: {pp1[:, 1].shape}')
    temp = pd.DataFrame(data={'user_id_hash': cur_uids.reshape(-1),
                              'user_purchase_binary_7_days': pp1[:,1],
                              'user_purchase_binary_14_days': pp2[:,1]})
    predict_df = pd.concat([predict_df, temp])
    run_ids = run_ids - set(cur_uids.reshape(-1))
    print(len(list(run_ids)))

2018-12-08: (20757,)
291811
2018-12-01: (12341,)
279470
2018-11-24: (34463,)
245007


In [68]:
predict_df.shape

(311574, 3)

In [69]:
left_out = list(set(submission.user_id_hash.unique()) -\
                set(predict_df.user_id_hash.unique()))

In [70]:
len(left_out)

994

In [71]:
predict_df.user_purchase_binary_7_days.median()

1.3107154472891125e-06

In [72]:
predict_df.user_purchase_binary_14_days.median()

3.745080448425142e-06

In [73]:
temp = pd.DataFrame(data={'user_id_hash': left_out,
                          'user_purchase_binary_7_days': np.zeros(len(left_out)),
                          'user_purchase_binary_14_days': np.zeros(len(left_out))})

In [74]:
predict_df = pd.concat([predict_df, temp])

In [75]:
predict_df.shape

(312568, 3)

In [77]:
submission.shape

(312568, 3)

In [78]:
predict_df.to_csv('../new_sub.csv', index=False, float_format='%.2f')

In [79]:
predict_df.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,0.000007,0.095824
1,000059859ec188af6035870faf885c3038cedda05b3a54...,0.000001,0.000006
2,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0.000002,0.000006
3,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,0.000001,0.000003
4,0000c5de50d243cc39f6ac4268d7511705585ca3f2a20f...,0.000001,0.000006
